In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

C:\Users\admin\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


2.9.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.2052 - accuracy: 0.6498
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 2s 15ms/step - loss: 1.1712 - accuracy: 0.6600 - val_loss: 0.7171 - val_accuracy: 0.7830
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4220 - accuracy: 0.8792
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.4264 - accuracy: 0.8790 - val_loss: 0.5314 - val_accuracy: 0.8380
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2729 - accuracy: 0.9329
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.2774 - accuracy: 0.9320 - val_loss: 0.4953 - val_accuracy: 0.8450
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.2127 - accuracy: 0.9516
Epoch 4: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 14ms/step - lo

In [5]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [6]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3407 - accuracy: 0.1080 - 237ms/epoch - 7ms/step
훈련되지 않은 모델의 정확도: 10.80%


In [7]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4195 - accuracy: 0.8650 - 108ms/epoch - 3ms/step
복원된 모델의 정확도: 86.50%


In [8]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2\cp-0005.ckpt

Epoch 10: saving model to training_2\cp-0010.ckpt

Epoch 15: saving model to training_2\cp-0015.ckpt

Epoch 20: saving model to training_2\cp-0020.ckpt

Epoch 25: saving model to training_2\cp-0025.ckpt

Epoch 30: saving model to training_2\cp-0030.ckpt

Epoch 35: saving model to training_2\cp-0035.ckpt

Epoch 40: saving model to training_2\cp-0040.ckpt

Epoch 45: saving model to training_2\cp-0045.ckpt

Epoch 50: saving model to training_2\cp-0050.ckpt


In [9]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index']

In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [11]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5057 - accuracy: 0.8770 - 240ms/epoch - 8ms/step
복원된 모델의 정확도: 87.70%


In [13]:
#수동으로 가중치 저장하기
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5057 - accuracy: 0.8770 - 240ms/epoch - 8ms/step
복원된 모델의 정확도: 87.70%


In [14]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 9ms/step - loss: 1.1553 - accuracy: 0.6870
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4097 - accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2733 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2057 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1636 - accuracy: 0.9660
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [15]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4353 - accuracy: 0.8630 - 192ms/epoch - 6ms/step
복원된 모델의 정확도: 86.30%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [17]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 7ms/step - loss: 1.1437 - accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4109 - accuracy: 0.8910
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2705 - accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1993 - accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1515 - accuracy: 0.9670


In [18]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4230 - accuracy: 0.8600 - 188ms/epoch - 6ms/step
복원된 모델의 정확도: 86.00%
